In [1]:
import sys
sys.path.append("C:\\Users\\KlimovaAnnaA\\Documents\\MyFiles\\Projects\\Working_attributes")
from Imports import * 


In [2]:
riskDebtAccounts_files = [x for x in os.listdir() if 'risk' in x and '.xl' in x and '~' not in x]
riskDebtAccounts_files

['riskDebtAccounts_(16).xlsx']

In [3]:
data_riskDebtAccounts = pd.read_excel(riskDebtAccounts_files[0], header=5)
data_riskDebtAccounts.head()

,active,action,type,nameIFRS,codeIFRS,sourceSystem,accountCode,accountPlanType,reportType,bscCode,bsxCode,overdueFlag,ocpFlag,resetOverdueFlag
0,Y,Include,Trade payables,Задолженность поставщикам и подрядчикам в евро,B601420,ETL document loader,NaN,NaN,AP,BSC300,BSX303,NaN,Y,NaN
1,Y,Include,Trade payables,Задолженность поставщикам и подрядчикам в браз...,B601420,ETL document loader,NaN,NaN,AP,BSC300,BSX304,NaN,Y,NaN
2,Y,Include,Trade payables,Задолженность поставщикам и подрядчикам в проч...,B601420,ETL document loader,NaN,NaN,AP,BSC300,BSX309,NaN,Y,NaN
3,Y,Include,Trade payables,Кредиторская задолженность за основные средств...,B601300,ETL document loader,NaN,NaN,AP,BSC300,BSX371,NaN,Y,NaN
4,Y,Include,Trade payables,НДС по кредиторской задолженности за основные ...,B769926,ETL document loader,NaN,NaN,AP,BSC300,BSX372,NaN,Y,NaN


In [45]:
data_riskDebtAccounts_work = data_riskDebtAccounts[:]
# xxmdg."riskDebtAccounts"."ocpFlag" = 'Y'
# xxmdg."riskDebtAccounts"."active" = 'Y'
# xxmdg."riskDebtAccounts"."action" = 'Include'
data_riskDebtAccounts_work = data_riskDebtAccounts_work[(data_riskDebtAccounts_work.ocpFlag=='Y')&(data_riskDebtAccounts_work.active=='Y')&(data_riskDebtAccounts_work.action=='Include')]
data_riskDebtAccounts_work = data_riskDebtAccounts_work[['type','sourceSystem','accountCode','reportType','bscCode']]
# pivots
pivot_1 = data_riskDebtAccounts_work.groupby(['bscCode','accountCode','reportType'],dropna=False,as_index=False)[['sourceSystem','type']].agg(lambda x: str(set(x)))
pivot_1.loc[(~pivot_1.accountCode.isna()),'accountCode_like'] = pivot_1.loc[(~pivot_1.accountCode.isna())&(pivot_1.accountCode.str.contains('%')),'accountCode'] # col for like statements
pivot_2 = pivot_1.groupby(['sourceSystem','reportType', 'type'],dropna=False,as_index=False)[['bscCode','accountCode','accountCode_like']].agg(lambda x: str(set(x)))
pivot_2 = pivot_2.replace({'{':'', '}':'', '\"':'','nan,':'', 'nan':np.nan}, regex=True)
# # filter to 'AR', 'AP' because of SQL error
# pivot_2 = pivot_2[(pivot_2.reportType=='\'AR\'')|(pivot_2.reportType=='\'AP\'')]
# pivot_2.head(5)
pivot_2

,sourceSystem,reportType,type,bscCode,accountCode,accountCode_like
0,"'OEBS12_EU', 'ETL document loader', 'OEBS12_SA'",AR,'Other receivables',"'BSC145', 'BSC140'",NaN,NaN
1,"'OEBS12_EU', 'ETL document loader', 'OEBS12_SA'",AR,'Trade receivables',"'BSC130', 'BSC135'",NaN,NaN
2,"'OEBS12_EU', 'ETL document loader', 'SAP_HRG',...",AP,'Other payables',"'BSC315', 'BSC310'",NaN,NaN
3,"'OEBS12_EU', 'ETL document loader', 'SAP_HRG',...",AP,'Provisions','BSC330',NaN,NaN
4,"'OEBS12_EU', 'ETL document loader', 'SAP_HRG',...",AP,'Trade payables',"'BSC300', 'BSC305'",NaN,NaN
5,'OEBS12_EU',AR,'Trade receivables','BSC130','015210',NaN
6,'SAP ECP',AP,'Other payables',NaN,"'5503030000', '5501010000', '5101010100', '550...",NaN
7,'SAP ECP',AP,'Provisions',NaN,"'2201910000', '2201910008'",NaN
8,'SAP ECP',AP,'Trade payables',NaN,"'5201011200', '5201010200', '5201010110', '520...",NaN
9,'SAP ECP',AR,'Other receivables',NaN,"'2603000100', '2601040000', '2603000000', '260...",NaN


In [50]:
pivot_2_SQL = pivot_2.copy()
# to sql statements
pivot_2_SQL.sourceSystem = pivot_2_SQL.sourceSystem.apply(lambda x: f'WHEN "sourceSystem" in ({x})')
pivot_2_SQL.reportType = pivot_2_SQL.reportType.apply(lambda x: f'and "reportType" = \'{x}\'')
pivot_2_SQL.loc[~pivot_2_SQL.bscCode.isna(), 'bscCode'] = pivot_2_SQL.loc[~pivot_2_SQL.bscCode.isna(), 'bscCode'].apply(lambda x: f'and "bscCode" in ({x})')
pivot_2_SQL.loc[~pivot_2_SQL.accountCode.isna(), 'accountCode'] = pivot_2_SQL.loc[~pivot_2_SQL.accountCode.isna(), 'accountCode'].apply(lambda x: f'and ("glAccount" in ({x})')
pivot_2_SQL.loc[~pivot_2_SQL.accountCode_like.isna(), 'accountCode_like'] = pivot_2_SQL.loc[~pivot_2_SQL.accountCode_like.isna(), 'accountCode_like'].apply(lambda x: f'or "glAccount" like {x})')
pivot_2_SQL.loc[(pivot_2_SQL.accountCode_like.isna())&(~pivot_2_SQL.accountCode.isna()), 'accountCode_like'] = ')'
pivot_2_SQL.type = pivot_2_SQL.type.apply(lambda x: f'THEN {x}')
# pivot_2_SQL
# to SQL
pivot_2_SQL = pivot_2_SQL[['sourceSystem','reportType','bscCode','accountCode','accountCode_like','type']]
case_statement = pivot_2_SQL.to_string(header=False, index=False, na_rep='')
### QUERY
query = f"""CASE
            {case_statement}
            END AS "accountPurpose"
            from RISKACCESS."unifiedARReport" 
            where "rowType" = 'M' 
"""
# to text
with open(f'{str(date.today())}_SQL_query.txt', 'w') as file:
    file.write(query)